In [1]:
import sys; sys.path.insert(0, '..') # So that we import the local copy of pyzx if you have installed from Github
import os
import csv

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

from multiprocessing import Pool
import pyzx as zx
from pyzx import cnot_mapper, architecture # Note that this is the local pyzx package from this repository/fork, not the one installed through pip or other means.
from pyzx import circuit
from pyzx import *
from pyzx.parity_maps import CNOT_tracker, build_random_parity_map
from pyzx.linalg import Mat2
from pyzx.simplify import full_reduce

Load Circuits for CombRowCol

In [2]:
comb_circuit_folder = "../circuits/combs-ZX-prop/"

n_qubits = [16]
n_cnots = [4,8,16,32,64,128,256,512,1024]
#n_non_cnots = [0,1,2,4,5,8,10,15,20]
#n_non_cnots = [0,4,10,20]
non_cnot_proportion = 0.15
loaded_circuits = {}
for qubits in n_qubits:
    for cnots in n_cnots:
        #for non_cnots in n_non_cnots:
        non_cnots = math.ceil(cnots * non_cnot_proportion)
        circuit_properties = {"Qubits"    : qubits,
                              "CNOTs"     : cnots,
                              "Non-CNOTs" : non_cnots}
        path = os.path.join(comb_circuit_folder,
                            str(circuit_properties["Qubits"])+"qubits", 
                            str(circuit_properties["CNOTs"])+"cnots", 
                            str(circuit_properties["Non-CNOTs"])+"non-cnots")
        loaded_circuits[f"({qubits},{cnots},{non_cnots})"] = []
        if os.path.exists(path):
            for file_name in os.listdir(path):
                loaded_circuits[f"({qubits},{cnots},{non_cnots})"].append(circuit.Circuit.from_qasm_file(os.path.join(path, file_name)))
        else:
            print("Directory Does Not Exist")

In [3]:
from IPython.display import display, Markdown

DEBUG = False
OUTER_DISPLAY = False
INNTER_DISPLAY = False

for qubits in n_qubits:
    #arch = architecture.create_architecture(architecture.SQUARE, n_qubits=qubits)
    arch = architecture.create_ibm_q20_tokyo_architecture()
    with open(f'COMBROWCOL - {arch.name}', mode='w') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')

        for cnots in n_cnots:
            non_cnots = math.ceil(cnots * non_cnot_proportion)
            #for non_cnots in n_non_cnots:
            equal = 0
            ratios = []
            for INDEX in range(5):
                if False:
                    # Get circuit
                    circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                    # Draw circuit
                    OUTER_DISPLAY and display(zx.draw(circ))
                    # Decompose circuit into comb and holes
                    decomposition = CombDecomposition.from_circuit(circ.copy())
                    comb = decomposition.comb
                    # Draw comb
                    OUTER_DISPLAY and display(zx.draw(comb))
                    # Recompose circuit from comb and holes
                    new_circuit = CombDecomposition.to_circuit(decomposition)
                    # Display recomposed circuit
                    OUTER_DISPLAY and display(zx.draw(new_circuit))
                    # Check whether the recomposed circuit is the same as the original
                    equality = new_circuit.verify_equality(circ)
                    equal += equality
                else:
                    circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                    new_circ, Id = combrowcol(circ, arch, DEBUG, OUTER_DISPLAY, INNTER_DISPLAY)
                    gate_ratio = (len(new_circ.gates) - non_cnots)/(len(circ.gates) - non_cnots) - 1
                    ratios.append(gate_ratio)
                    equality = new_circ.verify_equality(circ)
                    equal += equality
                    #print(f"INDEX: {INDEX} , GATE RATIO : {gate_ratio}")
                    #print(f"INDEX: {INDEX} {equality}")
                    #equal += Id
                    #print(f"Reduced to identity : {INDEX} {Id}")
            print(f"Qubits : {qubits}, CNOTs : {cnots}, Non-CNOTs : {non_cnots} -> {equal}, Ratio : {sum(ratios)/len(ratios)}")
            writer.writerow([qubits, cnots, non_cnots, sum(ratios)/len(ratios)])

Qubits : 16, CNOTs : 4, Non-CNOTs : 1 -> 5, Ratio : 3.0
Qubits : 16, CNOTs : 8, Non-CNOTs : 2 -> 5, Ratio : 3.875
Qubits : 16, CNOTs : 16, Non-CNOTs : 3 -> 5, Ratio : 5.0125
Qubits : 16, CNOTs : 32, Non-CNOTs : 5 -> 5, Ratio : 3.65
Qubits : 16, CNOTs : 64, Non-CNOTs : 10 -> 5, Ratio : 2.73125
Qubits : 16, CNOTs : 128, Non-CNOTs : 20 -> 5, Ratio : 1.93125
Qubits : 16, CNOTs : 256, Non-CNOTs : 39 -> 5, Ratio : 1.69140625
Qubits : 16, CNOTs : 512, Non-CNOTs : 77 -> 5, Ratio : 1.408984375
Qubits : 16, CNOTs : 1024, Non-CNOTs : 154 -> 5, Ratio : 1.3962890625


Cut and Route Method

In [4]:
def subcircuits(circuit):
    sub_circuits = []
    for gate in circuit.gates:
        if len(sub_circuits) == 0:
            sub_circuits.append([gate])
        else:
            current_name = sub_circuits[-1][0].name
            if current_name == "CNOT":
                if gate.name == "CNOT":
                    sub_circuits[-1].append(gate)
                else:
                    sub_circuits.append([gate])
            else:
                if gate.name == "CNOT":
                    sub_circuits.append([gate])
                else:
                    sub_circuits[-1].append(gate)
    return sub_circuits

def cutrowcol(circuit, arch):
    sub_circuits = subcircuits(circ.copy())
    new_circuit = Circuit(circuit.qubits)
    for sub_circuit in sub_circuits:
        if sub_circuit[0].name != "CNOT":
            new_circuit.gates = new_circuit.gates + sub_circuit
        else:
            # Get matrix corresponding to CNOT sub circuit
            cnottracker = CNOT_tracker(circuit.qubits)
            cnottracker.gates = sub_circuit
            cnottracker.update_matrix()
            temp_circuit = CNOT_tracker(circuit.qubits)
            # rowcol reduced the matrix of cnottracker to the identity under the constraints of
            # arch, logging each row operation at a CNOT in temp_circuit
            rowcol(cnottracker.matrix, arch, temp_circuit, debug=None)
            # The gates of temp_circuit are then added to the new_circuit
            new_circuit.gates = new_circuit.gates + temp_circuit.gates
    return new_circuit
    

In [5]:
from IPython.display import display, Markdown

DEBUG = False
OUTER_DISPLAY = False
INNTER_DISPLAY = False


for qubits in n_qubits:
    #arch = architecture.create_architecture(architecture.SQUARE, n_qubits=qubits)
    arch = architecture.create_ibm_q20_tokyo_architecture()
    with open(f'CUTROWCOL - {arch.name}', mode='w') as csv_file:
        #fieldnames = ['Number of Qubits', 'Number of CNOT Gates', 'Number of Non CNOT Gates']
        writer = csv.writer(csv_file, delimiter=',')

        for cnots in n_cnots:
            non_cnots = math.ceil(cnots * non_cnot_proportion)
            #for non_cnots in n_non_cnots:
            equal = 0
            ratios = []
            for INDEX in range(20):
                if False:
                    circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                    print(f"Gates : {circ.gates}")
                    print(f"Sub Circuits : {subcircuits(circ)}")
                else:
                    circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]                                        
                    new_circ = cutrowcol(circ, arch)
                    gate_ratio = (len(new_circ.gates) - non_cnots)/(len(circ.gates) - non_cnots) - 1
                    ratios.append(gate_ratio)
                    equality = new_circ.verify_equality(circ)
                    equal += equality
                    #print(f"INDEX: {INDEX} , GATE RATIO : {gate_ratio}")
                    #print(f"INDEX: {INDEX} {equality}")
            print(f"Qubits : {qubits}, CNOTs : {cnots}, Non-CNOTs : {non_cnots} -> {equal}, Ratio : {sum(ratios)/len(ratios)}")
            writer.writerow([qubits, cnots, non_cnots, sum(ratios)/len(ratios)])

Qubits : 16, CNOTs : 4, Non-CNOTs : 1 -> 20, Ratio : 5.075
Qubits : 16, CNOTs : 8, Non-CNOTs : 2 -> 20, Ratio : 4.05
Qubits : 16, CNOTs : 16, Non-CNOTs : 3 -> 20, Ratio : 4.5125
Qubits : 16, CNOTs : 32, Non-CNOTs : 5 -> 20, Ratio : 4.1
Qubits : 16, CNOTs : 64, Non-CNOTs : 10 -> 20, Ratio : 4.08515625
Qubits : 16, CNOTs : 128, Non-CNOTs : 20 -> 20, Ratio : 4.13515625
Qubits : 16, CNOTs : 256, Non-CNOTs : 39 -> 20, Ratio : 4.1484375
Qubits : 16, CNOTs : 512, Non-CNOTs : 77 -> 20, Ratio : 4.1310546875
Qubits : 16, CNOTs : 1024, Non-CNOTs : 154 -> 20, Ratio : 4.050634765625
